In [21]:
import cv2
import numpy as np

from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy.ndimage
import numpy as np
import os
from skimage.transform import resize

In [30]:
folder_path = "../Expirements/SavedTransforms/0-99_usual"
frames_path = os.path.join(folder_path, 'frames')
file_list = []

for file in os.listdir(frames_path):
    if file.endswith(".png"):
        file_list.append(os.path.join(frames_path, file))
file_list.sort()

print(len(file_list))


27


In [31]:
T = len(file_list)

crops = []
for idx in tqdm(range(T)):
    crops.append(255.0*mpimg.imread(file_list[idx])[:,:,:3])

In [32]:
writer = cv2.VideoWriter(folder_path + '/report/optimization.avi', cv2.VideoWriter_fourcc(*'MJPG'), 5, (851, 829), True)
for i in range(len(crops)):
    #x = np.random.randint(255, size=(829, 851, 3)).astype('uint8')
    x = crops[i][:,:,::-1].astype('uint8')
    writer.write(x)
writer.release()